In [4]:
import bolift
import pandas as pd
import numpy as np

In [8]:
df = pd.read_csv('dataset.tsv', sep='\t')
df

,Temperatur,DOP,Time,Wavelength,Intensity
0,0,10,1h,440.0,66
1,0,10,1h,441.0,54
2,0,10,1h,442.0,72
3,0,10,1h,443.0,69
4,0,10,1h,444.0,80
...,...,...,...,...,...
10464,0,10,10m,796.0,92
10465,0,10,10m,797.0,164
10466,0,10,10m,798.0,138
10467,0,10,10m,799.0,90


In [ ]:
prompt_template = ""

In [ ]:
asktell = bolift.AskTellFewShotTopk(
    prefix="The following are correctly answered questions about H2 production by Shewanella oneidensis MR-1 and quantum dots. " \
          " Each answer should be numeric and ends with ###",
    x_formatter=lambda x: f"the experimental procedure: {x}",
    y_name="H2 production",
    y_formatter=lambda y: f"{y:.2f}",
    model="gpt-4",
    selector_k=5,
    temperature=0.7
)